In [817]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function


import datetime
#imports

In [818]:
batters_df = pd.read_csv('../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../data/individual_pitching_stats.csv')
misc_batters = pd.read_csv('../data/batters_teams.csv')
misc_pitchers = pd.read_csv('../data/pitchers_teams.csv')

In [819]:
bat_xbadiff = batters_df[['last_name', 'first_name', 'xbadiff','batting_avg', 'xba']]
#Make a table with just the xBA difference

In [820]:
pitch_xbadiff = pitchers_df[['last_name', 'first_name', 'xbadiff', 'batting_avg', 'xba']]
# pitch_xbadiff.sort_values('xba', ascending=False)

# Probable Pitchers

In [821]:
url = 'https://www.mlb.com/probable-pitchers/2022-04-29'
#URL of where I will get the pitchers' names

In [822]:
res = requests.get(url)
soup = BeautifulSoup(res.content)
#Making request to the site and storing it in soup

In [823]:
pitchers = soup.find_all('a', {'class': 'probable-pitchers__pitcher-name-link'})
probable_pitchers = []
for i in pitchers:
    probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [824]:
probable_pitchers[0]

'<a class="probable-pitchers__pitcher-name-link" href="/player/yu-darvish-506433">Yu Darvish</a>'

In [825]:
probable_pitchers = [i.split('">')[1] for i in probable_pitchers] 
#Getting only the pitchers' names

In [826]:
probable_pitchers = [i.split('</a')[0] for i in probable_pitchers] 
#Getting only the pitchers' names part 2

In [827]:
probable_pitchers[0]

'Yu Darvish'

In [828]:
probable_pitchers = [i.split() for i in probable_pitchers]
#Splitting name into first and last name columns

In [829]:
df_probable_pitchers = pd.DataFrame(probable_pitchers)
#Setting list as Pandas DF 

In [830]:
df_probable_pitchers = df_probable_pitchers.rename(columns={0: "first_name", 1: "last_name"})
#Renaming column

# Get Pitchers' Teams

In [831]:
misc_pitchers = misc_pitchers[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [832]:
misc_pitchers[['first_name', 'last_name']] = misc_pitchers['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [833]:
misc_pitchers = misc_pitchers.drop(columns='Player')
#We don't need this full name column anymore

In [834]:
misc_pitchers = misc_pitchers[['last_name', 'first_name', 'Team']]
#reordering columns

# Get Batters' Teams

In [835]:
misc_batters = misc_batters[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [836]:
misc_batters[['first_name', 'last_name']] = misc_batters['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [837]:
misc_batters = misc_batters.drop(columns='Player')
#We don't need this full name column anymore

In [838]:
misc_batters = misc_batters[['last_name', 'first_name', 'Team']]
#Reordering columns

In [839]:
df_probable_pitchers.head()

,first_name,last_name
0,Yu,Darvish
1,Zach,Thompson
2,Matt,Brash
3,Elieser,Hernandez
4,Rich,Hill


In [843]:
pitchers_df['first_name'] = pitchers_df['first_name'].str.strip()
batters_df['first_name'] = batters_df['first_name'].str.strip()
#Removing empty space in these column entries for example this makes ' Cabrera' => 'Cabrera'

batters_df and pitchers_df front names all have a space in front of them and that's why merge is not working

# Attaching according Team Name to each player in all DFs

The datasets on Baseball Savant do not include the team name with they players, so I will add them in

In [844]:
df_probable_pitchers = pd.merge(df_probable_pitchers, misc_pitchers, on=['last_name', 'first_name'], how='inner')
pitchers_df = pd.merge(pitchers_df, misc_pitchers, on=['last_name', 'first_name'], how='inner')
batters_df = pd.merge(batters_df, misc_batters, on=['last_name', 'first_name'], how='inner')